# Preamble

In [9]:
import sys
# pandas to manipulate SQL answer set
import pandas as pd
import numpy as np
import time,os
# for Posgresql and other RDBMS
from sqlalchemy import create_engine,event,schema,Table,Column, Integer, Float, String, MetaData, TIMESTAMP, Date, text, inspect
from sqlalchemy_utils import database_exists,create_database
from sqlalchemy.orm import sessionmaker
from sqlalchemy.schema import CreateTable

## Connect to the DB server

In [3]:
# Define your PostgreSQL server connection details
database_name = 'group_7_2024'
username = 'group_7_2024'
password = 'SNMsELrjkpQC'
host = 'dbcourse.cs.aalto.fi'  # or your server address
port = '5432'  # default PostgreSQL port


# Create a connection URL
connection_url = f'postgresql+psycopg2://{username}:{password}@{host}:{port}/{database_name}'

try:
    # Create an engine to test the connection
    engine = create_engine(connection_url) #.replace(f'/{database_name}', '/postgres'))  # Connect to the default 'postgres' database
    with engine.connect() as conn:
        print("Connection to PostgreSQL server successful.")
except Exception as e:
    print(f"Failed to connect to the PostgreSQL server: {e}")
    exit(1)

# Check if the database already exists
if not database_exists(connection_url):
    # Create the new database
    create_database(connection_url)
    print(f"Database '{database_name}' created successfully.")
else:
    print(f"Database '{database_name}' already exists.")



Connection to PostgreSQL server successful.
Database 'group_7_2024' already exists.


## Check the DB content

In [4]:
# Create an inspector
inspector = inspect(engine)

# Get the list of all tables
tables = inspector.get_table_names()

# Print the tables
print("Tables in the database:")
for table in tables:
    print(table)

Tables in the database:
city
volunteer
volunteer_range
skill
skill_assignment
interest
interest_assignment
beneficiary
request
request_skill
volunteer_application
request_location


## d) Analysis

(10p) Create your own scoring system to calculate the matching percentage
from all the attributes of a volunteer that you find relevant: e.g: interest, travel
readiness, volunteer range, number of skill matches, etc. Make a compelling
case for your scoring scheme and suggest a top 5 candidates for each
request according to this system. Does it match the candidates you have
found in past questions?

 First, let's cut off all applicants whose application is invalid or obsolete.

 Then, as point of reference for the scoring system, let's define an ideal candidate:
 - has the best match of skills w.r.t. their value 
 - lives the closest to the location where they're needed (the closest one is always considered)
 - whose readiness to travel is the shortest

Hence, for each candidate we compute a score made of these three components:
- skill matching score skill_score = total value of matching skills
- distance to target distance_penalty = Euclidean distance between the applicant location and the nearest target location of the request (matching with volunteer range)
- readiness to travel penalty delay_penalty = readiness to travel scaled to days

Thus the final applicant score = a * skill_score - b * distance_penalty - c * delay_penalty.
This setup allows beneficiaries tune the weight parameters [a, b, c] for the respective score components as per their preferences. 

In [5]:
# check the total number of valid applications (make sure that the target location matches the volunteer range)
my_query = text(f"""
select distinct va.id
from volunteer_application va, request_location rl, volunteer_range vr 
where va.is_valid = true and va.volunteer_id = vr.volunteer_id  and va.request_id = rl.request_id  and rl.city_id = vr.city_id;
""")

valid_applications = pd.read_sql_query(my_query, con=engine.connect())
len(valid_applications)

1503

### Get valid applicants and their travel readiness (scaled to days)

In [6]:
# check the total number of valid applications (make sure that the target location matches the volunteer range)
my_query = text(f"""
select distinct valid_applicants.request_id as req_id, valid_applicants.volunteer_id, v.travel_readiness
from volunteer v,
	(select va.volunteer_id, va.request_id
	from volunteer_application va, request_location rl, volunteer_range vr 
	where va.is_valid = true and va.volunteer_id = vr.volunteer_id  and va.request_id = rl.request_id  and rl.city_id = vr.city_id) as valid_applicants
where v.id = valid_applicants.volunteer_id
order by valid_applicants.request_id, valid_applicants.volunteer_id;            
""")

valid_applicants = pd.read_sql_query(my_query, con=engine.connect())
MINS_PER_DAY = 1440
valid_applicants["travel_readiness"] = valid_applicants["travel_readiness"] / MINS_PER_DAY
valid_applicants

,req_id,volunteer_id,travel_readiness
0,1,011074-9149,0.318056
1,1,160903A941P,0.733333
2,1,210753-990T,0.675000
3,1,211074-9401,0.338194
4,1,211099-910H,0.886806
...,...,...,...
1498,381,211099-910H,0.886806
1499,381,220857-9810,0.363889
1500,382,100396-906V,0.538889
1501,382,200472-937X,0.827778


### Compute the skill score for each request and every applicant

In [7]:
# this query picks up valid applicants, then for each request and for every applicant sums up the value of their skills (if they match those of the request)
my_query = text(f"""
select biggy.req_id, biggy.volunteer_id, sum(v_val) as skill_score
from volunteer v, volunteer_range vr, request_location rl, 
(select r.id as req_id, va.volunteer_id, sa.skill_name as v_skill, rs.skill_name as neeeded_skill, rs.value, 
case when sa.skill_name = rs.skill_name then rs.value else 0 end as v_val
FROM request r
JOIN volunteer_application va ON va.request_id = r.id
LEFT JOIN request_skill rs ON rs.request_id = r.id 
LEFT JOIN skill_assignment sa ON sa.volunteer_id = va.volunteer_id AND sa.skill_name = rs.skill_name 
WHERE va.is_valid = true and (r.end_date > va.modified)
ORDER BY r.id, volunteer_id
) as biggy
where v.id  = biggy.volunteer_id and v.id = vr.volunteer_id and vr.city_id = rl.city_id and biggy.req_id = rl.request_id
GROUP BY biggy.req_id, biggy.volunteer_id
order by biggy.req_id, biggy.volunteer_id;
""")

skill_score_df = pd.read_sql_query(my_query, con=engine.connect())
skill_score_df


,req_id,volunteer_id,skill_score
0,1,011074-9149,4.0
1,1,160903A941P,1.0
2,1,210753-990T,0.0
3,1,211074-9401,4.0
4,1,211099-910H,4.0
...,...,...,...
1498,381,211099-910H,13.0
1499,381,220857-9810,18.0
1500,382,100396-906V,12.0
1501,382,200472-937X,6.0


### Compute the distance penalty

### Compute the skill score for each request and every applicant

In [10]:
my_query = text(f"""
select valid_applicants.request_id as req_id, valid_applicants.volunteer_id, v.city_id as home_id, v.geolocation as home_geo, valid_applicants.target_city, c.geolocation as target_geo
from city c, 
	(select vol.id, vol.city_id, c2.geolocation 
	 from volunteer vol, city c2 
	 where vol.city_id = c2.id
	) v,
	-- get valid applicants
	(select distinct rl.request_id, va.volunteer_id, rl.city_id as target_city
	from volunteer_application va, request_location rl, volunteer_range vr 
	where va.is_valid = true and va.volunteer_id = vr.volunteer_id  and va.request_id = rl.request_id  and rl.city_id = vr.city_id) as valid_applicants
where (c.id = valid_applicants.target_city) and (v.id = valid_applicants.volunteer_id) 
ORDER BY valid_applicants.request_id, valid_applicants.volunteer_id;
""")

target_home_df = pd.read_sql_query(my_query, con=engine.connect())
target_home_df["home_geo"] = target_home_df["home_geo"].apply(lambda r: np.array(list(map(float, r.split("/")))))
target_home_df["target_geo"] = target_home_df["target_geo"].apply(lambda r: np.array(list(map(float, r.split("/")))))
target_home_df["distance"] = target_home_df["home_geo"] - target_home_df["target_geo"]
target_home_df["distance"] = target_home_df["distance"].apply(lambda r: np.linalg.norm(r))
target_home_df


,req_id,volunteer_id,home_id,home_geo,target_city,target_geo,distance
0,1,011074-9149,704,"[13.4134995, 45.79265]",504,"[45.475847, -105.227997]",154.386625
1,1,011074-9149,704,"[13.4134995, 45.79265]",704,"[13.4134995, 45.79265]",0.000000
2,1,160903A941P,72,"[-79.132557, -40.995129]",687,"[66.6004235, -62.724453]",147.344037
3,1,210753-990T,426,"[47.244262, 80.880444]",504,"[45.475847, -105.227997]",186.116843
4,1,210753-990T,426,"[47.244262, 80.880444]",426,"[47.244262, 80.880444]",0.000000
...,...,...,...,...,...,...,...
2088,382,100396-906V,426,"[47.244262, 80.880444]",426,"[47.244262, 80.880444]",0.000000
2089,382,200472-937X,834,"[37.93552, 12.226293]",687,"[66.6004235, -62.724453]",80.245193
2090,382,200472-937X,834,"[37.93552, 12.226293]",72,"[-79.132557, -40.995129]",128.598034
2091,382,200472-937X,834,"[37.93552, 12.226293]",886,"[18.6986795, -17.160223]",35.122975


In [12]:
distance_df = pd.DataFrame(columns=[["req_id",	"volunteer_id", "home_target_distance"]])

ind = 0
for req in target_home_df["req_id"].unique():
   this_req_df = target_home_df[target_home_df["req_id"] == req]
   for volunteer in this_req_df["volunteer_id"].unique():      
      this_vol_df = this_req_df[this_req_df["volunteer_id"] == volunteer]
      distance_df.loc[ind, "req_id"] = req
      distance_df.loc[ind, "volunteer_id"] = volunteer
      distance_df.loc[ind, "home_target_distance"] = this_vol_df["distance"].min()
      ind += 1

distance_df["req_id"] = distance_df["req_id"].astype(int)
distance_df.reset_index(drop=True)
distance_df

,req_id,volunteer_id,home_target_distance
0,1,011074-9149,0.0
1,1,160903A941P,147.344037
2,1,210753-990T,0.0
3,1,211074-9401,0.0
4,1,211099-910H,0.0
...,...,...,...
1498,381,211099-910H,0.0
1499,381,220857-9810,0.0
1500,382,100396-906V,0.0
1501,382,200472-937X,35.122975


### Compute total score

In [13]:
imprtance_weights = np.array([10.0, 0.1, 1.0])

# merge individual score components into a single table
total_score = skill_score_df.copy()
total_score["home_target_distance"] = distance_df["home_target_distance"].astype(float)
total_score["travel_readiness"] = valid_applicants["travel_readiness"].astype(float)

# compute total score as a linear combination of its components
total_score["total_score"] = total_score["skill_score"] * imprtance_weights[0] - total_score["home_target_distance"] * imprtance_weights[1] - total_score["travel_readiness"] * imprtance_weights[2] 
total_score = total_score.sort_values(by=['req_id', 'total_score'], ascending=[True, False]) # sort the results by total_score
total_score

,req_id,volunteer_id,skill_score,home_target_distance,travel_readiness,total_score
0,1,011074-9149,4.0,0.000000,0.318056,39.681944
3,1,211074-9401,4.0,0.000000,0.338194,39.661806
4,1,211099-910H,4.0,0.000000,0.886806,39.113194
5,1,250681-919H,1.0,0.000000,0.101389,9.898611
2,1,210753-990T,0.0,0.000000,0.675000,-0.675000
...,...,...,...,...,...,...
1495,381,150400A944B,0.0,0.000000,0.369444,-0.369444
1494,381,140974-9866,0.0,47.463660,0.865972,-5.612338
1500,382,100396-906V,12.0,0.000000,0.538889,119.461111
1501,382,200472-937X,6.0,35.122975,0.827778,55.659925


### For each request keep only top 5

In [14]:
top5_per_req = pd.DataFrame(columns=total_score.columns)

for req in total_score["req_id"].unique():
    top5_per_req = top5_per_req.append(total_score[total_score["req_id"] == req].head(5), ignore_index=True)
        
top5_per_req.head(25)

C:\Users\juriv\AppData\Local\Temp\ipykernel_16028\2133389450.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top5_per_req = top5_per_req.append(total_score[total_score["req_id"] == req].head(5), ignore_index=True)


,req_id,volunteer_id,skill_score,home_target_distance,travel_readiness,total_score
0,1,011074-9149,4.0,0.000000,0.318056,39.681944
1,1,211074-9401,4.0,0.000000,0.338194,39.661806
2,1,211099-910H,4.0,0.000000,0.886806,39.113194
3,1,250681-919H,1.0,0.000000,0.101389,9.898611
4,1,210753-990T,0.0,0.000000,0.675000,-0.675000
5,2,270794-9576,24.0,92.048626,0.084028,230.711110
6,2,220782-910B,16.0,91.053993,0.361111,150.533490
7,2,200569-926L,5.0,0.000000,0.073611,49.926389
8,3,100494-989U,4.0,92.048626,0.586806,30.208332
9,3,220782-910B,0.0,126.873524,0.361111,-13.048464


## Close connection to the DB

In [15]:
# Dispose of the engine to close the connection
engine.dispose()
print("Connection to PostgreSQL server closed.")

Connection to PostgreSQL server closed.
